In [2]:
import cv2
import numpy as np
import pandas as pd
import os
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model
best_model = YOLO('yolov8n.pt')

# Define the traffic intensity thresholds
low_threshold = 4
high_threshold = 10

# Open the video
cap = cv2.VideoCapture('l1.mp4')

# Get frame dimensions
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Create a directory for processed frames if it doesn't exist
os.makedirs('processed_frames', exist_ok=True)

# Define the area of interest (AOI) with a starting height of 40%
aoi_top = int(frame_height * 0.40)  # Start from 40% of the frame height
aoi_bottom = frame_height

# Define the height of the AOI
aoi_height = aoi_bottom - aoi_top  # This calculates the height of the AOI

# Define the left and right division within the AOI
left_aoi_x_start = 0
left_aoi_x_end = frame_width // 2
right_aoi_x_start = frame_width // 2
right_aoi_x_end = frame_width

# Define font, scale, and colors for the annotations
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5  # Reduced font size
font_color = (255, 255, 255)  # Default color for text

# Create lists to store vehicle counts and traffic intensities
frame_data = []

# Read until the video is completed
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret:
        # Create a copy of the original frame to modify
        detection_frame = frame.copy()
        
        # Perform inference on the frame
        results = best_model.predict(detection_frame, imgsz=640, conf=0.4)
        processed_frame = results[0].plot(line_width=1)

        # Draw the area of interest (AOI) rectangle
        cv2.rectangle(processed_frame, (left_aoi_x_start, aoi_top), (left_aoi_x_end, aoi_bottom), (0, 255, 0), 2)
        cv2.rectangle(processed_frame, (right_aoi_x_start, aoi_top), (right_aoi_x_end, aoi_bottom), (255, 0, 0), 2)

        # Retrieve the bounding boxes from the results
        bounding_boxes = results[0].boxes

        # Initialize counters for vehicles in each AOI
        vehicles_in_left_aoi = 0
        vehicles_in_right_aoi = 0

        # Loop through each bounding box to count vehicles in each AOI
        for box in bounding_boxes.xyxy:
            # Check if the vehicle is in the area of interest based on the bounding box
            if box[1] > aoi_top and box[3] < aoi_bottom:  # Check vertical position
                if box[0] < left_aoi_x_end:  # Check left AOI
                    vehicles_in_left_aoi += 1
                elif box[0] >= right_aoi_x_start:  # Check right AOI
                    vehicles_in_right_aoi += 1

        # Determine the traffic intensity for the left AOI
        if vehicles_in_left_aoi == 0:
            traffic_intensity_left = "None"
            font_color_left = (255, 255, 255)  # White for None
        elif 1 <= vehicles_in_left_aoi <= low_threshold:
            traffic_intensity_left = "Mild"
            font_color_left = (0, 165, 255)  # Orange for Mild
        else:  # For 6 vehicles and above
            traffic_intensity_left = "Strong"
            font_color_left = (0, 0, 255)  # Red for Strong

        # Determine the traffic intensity for the right AOI
        if vehicles_in_right_aoi == 0:
            traffic_intensity_right = "None"
            font_color_right = (255, 255, 255)  # White for None
        elif 1 <= vehicles_in_right_aoi <= low_threshold:
            traffic_intensity_right = "Mild"
            font_color_right = (0, 165, 255)  # Orange for Mild
        else:  # For 6 vehicles and above
            traffic_intensity_right = "Strong"
            font_color_right = (0, 0, 255)  # Red for Strong

        # Add annotations for left lane vehicle count and traffic intensity
        cv2.putText(processed_frame, f'Vehicles in Left AOI: {vehicles_in_left_aoi}', 
                    (10, 50), font, font_scale, font_color_left, 2, cv2.LINE_AA)
        cv2.putText(processed_frame, f'Traffic Intensity: {traffic_intensity_left}', 
                    (10, 100), font, font_scale, font_color_left, 2, cv2.LINE_AA)

        # Add annotations for right lane vehicle count and traffic intensity
        text_left_aoi_count = f'Vehicles in Right AOI: {vehicles_in_right_aoi}'
        text_traffic_intensity = f'Traffic Intensity: {traffic_intensity_right}'

        # Calculate the width of the text to properly align it to the right
        text_size_count = cv2.getTextSize(text_left_aoi_count, font, font_scale, 2)[0]
        text_size_intensity = cv2.getTextSize(text_traffic_intensity, font, font_scale, 2)[0]

        # Positioning the text for right AOI on the right side
        cv2.putText(processed_frame, text_left_aoi_count, 
                    (frame_width - text_size_count[0] - 10, 50), font, font_scale, font_color_right, 2, cv2.LINE_AA)
        cv2.putText(processed_frame, text_traffic_intensity, 
                    (frame_width - text_size_intensity[0] - 10, 100), font, font_scale, font_color_right, 2, cv2.LINE_AA)

        # Store data for the current frame
        frame_data.append({
            'Frame': cap.get(cv2.CAP_PROP_POS_FRAMES),
            'Left_AOI_Count': vehicles_in_left_aoi,
            'Left_AOI_Intensity': traffic_intensity_left,
            'Right_AOI_Count': vehicles_in_right_aoi,
            'Right_AOI_Intensity': traffic_intensity_right
        })

        # Save the processed frame to the processed_frames directory
        frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  # Get the current frame number
        cv2.imwrite(f'processed_frames/frame_{frame_number}.jpg', processed_frame)  # Save the frame as an image

        # Display the processed frame
        cv2.imshow('Real-time Traffic Analysis', processed_frame)

        # Press Q on the keyboard to exit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Convert frame data to a DataFrame
frame_df = pd.DataFrame(frame_data)

# Save the frame data to a CSV file
frame_df.to_csv('traffic_analysis_results_two_way.csv', index=False)

# Release the video capture and video write objects
cap.release()

# Close all the frames
cv2.destroyAllWindows()



0: 384x640 1 car, 66.8ms
Speed: 0.0ms preprocess, 66.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 58.2ms
Speed: 3.0ms preprocess, 58.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 61.0ms
Speed: 2.0ms preprocess, 61.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 55.7ms
Speed: 2.0ms preprocess, 55.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 59.9ms
Speed: 0.0ms preprocess, 59.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 51.1ms
Speed: 2.0ms preprocess, 51.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 53.3ms
Speed: 2.0ms preprocess, 53.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 46.2ms
Speed: 2.0ms preprocess, 46.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 

In [1]:
import cv2
import numpy as np
import pandas as pd
import os
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model
best_model = YOLO('yolov8n.pt')

# Define the traffic intensity thresholds
low_threshold = 4
high_threshold = 10

# Open the video
cap = cv2.VideoCapture('l1.mp4')

# Get frame dimensions
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Create a directory for processed frames if it doesn't exist
os.makedirs('processed_frames', exist_ok=True)

# Define the area of interest (AOI) with a starting height of 40%
aoi_top = int(frame_height * 0.40)  # Start from 40% of the frame height
aoi_bottom = frame_height

# Define the left and right division within the AOI
left_aoi_x_start = 0
left_aoi_x_end = frame_width // 2
right_aoi_x_start = frame_width // 2
right_aoi_x_end = frame_width

# Define font, scale, and colors for the annotations
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5  # Reduced font size
font_color = (255, 255, 255)  # Default color for text

# Create a list to store vehicle counts and traffic intensities
frame_data = []

# Read until the video is completed
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Create a copy of the original frame to modify
    detection_frame = frame.copy()

    # Perform inference on the frame
    results = best_model.predict(detection_frame, imgsz=640, conf=0.4)
    processed_frame = results[0].plot(line_width=1)

    # Draw the area of interest (AOI) rectangle
    cv2.rectangle(processed_frame, (left_aoi_x_start, aoi_top), (left_aoi_x_end, aoi_bottom), (0, 255, 0), 2)
    cv2.rectangle(processed_frame, (right_aoi_x_start, aoi_top), (right_aoi_x_end, aoi_bottom), (255, 0, 0), 2)

    # Retrieve the bounding boxes from the results
    bounding_boxes = results[0].boxes

    # Initialize counters for vehicles in each AOI
    vehicles_in_left_aoi = 0
    vehicles_in_right_aoi = 0

    # Loop through each bounding box to count vehicles in each AOI
    for box in bounding_boxes.xyxy:
        # Check if the vehicle is in the area of interest based on the bounding box
        if box[1] > aoi_top and box[3] < aoi_bottom:  # Check vertical position
            if box[0] < left_aoi_x_end:  # Check left AOI
                vehicles_in_left_aoi += 1
            elif box[0] >= right_aoi_x_start:  # Check right AOI
                vehicles_in_right_aoi += 1

    # Determine the traffic intensity for the left AOI
    if vehicles_in_left_aoi == 0:
        traffic_intensity_left = "None"
        font_color_left = (255, 255, 255)  # White for None
    elif 1 <= vehicles_in_left_aoi <= low_threshold:
        traffic_intensity_left = "Mild"
        font_color_left = (0, 165, 255)  # Orange for Mild
    else:  # For 6 vehicles and above
        traffic_intensity_left = "Strong"
        font_color_left = (0, 0, 255)  # Red for Strong

    # Determine the traffic intensity for the right AOI
    if vehicles_in_right_aoi == 0:
        traffic_intensity_right = "None"
        font_color_right = (255, 255, 255)  # White for None
    elif 1 <= vehicles_in_right_aoi <= low_threshold:
        traffic_intensity_right = "Mild"
        font_color_right = (0, 165, 255)  # Orange for Mild
    else:  # For 6 vehicles and above
        traffic_intensity_right = "Strong"
        font_color_right = (0, 0, 255)  # Red for Strong

    # Add annotations for left lane vehicle count and traffic intensity
    cv2.putText(processed_frame, f'Vehicles in Left AOI: {vehicles_in_left_aoi}', 
                (10, 50), font, font_scale, font_color_left, 2, cv2.LINE_AA)
    cv2.putText(processed_frame, f'Traffic Intensity: {traffic_intensity_left}', 
                (10, 100), font, font_scale, font_color_left, 2, cv2.LINE_AA)

    # Add annotations for right lane vehicle count and traffic intensity
    text_right_aoi_count = f'Vehicles in Right AOI: {vehicles_in_right_aoi}'
    text_traffic_intensity = f'Traffic Intensity: {traffic_intensity_right}'

    # Calculate the width of the text to properly align it to the right
    text_size_count = cv2.getTextSize(text_right_aoi_count, font, font_scale, 2)[0]
    text_size_intensity = cv2.getTextSize(text_traffic_intensity, font, font_scale, 2)[0]

    # Positioning the text for right AOI on the right side
    cv2.putText(processed_frame, text_right_aoi_count, 
                (frame_width - text_size_count[0] - 10, 50), font, font_scale, font_color_right, 2, cv2.LINE_AA)
    cv2.putText(processed_frame, text_traffic_intensity, 
                (frame_width - text_size_intensity[0] - 10, 100), font, font_scale, font_color_right, 2, cv2.LINE_AA)

    # Store data for the current frame
    frame_data.append({
        'Frame': cap.get(cv2.CAP_PROP_POS_FRAMES),
        'Left_AOI_Count': vehicles_in_left_aoi,
        'Left_AOI_Intensity': traffic_intensity_left,
        'Right_AOI_Count': vehicles_in_right_aoi,
        'Right_AOI_Intensity': traffic_intensity_right
    })

    # Save the processed frame to the processed_frames directory
    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  # Get the current frame number
    cv2.imwrite(f'processed_frames/frame_{frame_number}.jpg', processed_frame)  # Save the frame as an image

    # Display the processed frame
    cv2.imshow('Real-time Traffic Analysis', processed_frame)

    # Press Q on the keyboard to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Convert frame data to a DataFrame
frame_df = pd.DataFrame(frame_data)

# Save the frame data to a CSV file
frame_df.to_csv('traffic_analysis_results_two_way.csv', index=False)

# Release the video capture and close all the frames
cap.release()
cv2.destroyAllWindows()



0: 384x640 1 car, 387.8ms
Speed: 18.0ms preprocess, 387.8ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 370.1ms
Speed: 10.0ms preprocess, 370.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 381.5ms
Speed: 13.0ms preprocess, 381.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 466.1ms
Speed: 8.0ms preprocess, 466.1ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 278.5ms
Speed: 15.0ms preprocess, 278.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 397.6ms
Speed: 20.3ms preprocess, 397.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 503.9ms
Speed: 8.9ms preprocess, 503.9ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 409.1ms
Speed: 3.4ms preprocess, 409.1ms inference, 4.0ms postprocess per image at shape (1, 3, 3